## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2022-07-19 22:00:19,clear sky,-23.1203,-134.9692,74.10,76,2,9.22
1,1,Kapaa,US,2022-07-19 21:57:32,broken clouds,22.0752,-159.3190,84.18,72,64,7.00
2,2,Tuktoyaktuk,CA,2022-07-19 22:00:20,overcast clouds,69.4541,-133.0374,45.23,92,100,10.58
3,3,Nouadhibou,MR,2022-07-19 22:00:21,scattered clouds,20.9310,-17.0347,67.30,95,26,16.53
4,4,Busselton,AU,2022-07-19 21:57:31,light rain,-33.6500,115.3333,59.59,81,100,12.39


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements 
# to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Kapaa,US,2022-07-19 21:57:32,broken clouds,22.0752,-159.3190,84.18,72,64,7.00
9,9,Tiarei,PF,2022-07-19 22:00:23,clear sky,-17.5333,-149.3333,76.46,67,4,2.64
13,13,Atuona,PF,2022-07-19 22:00:25,clear sky,-9.8000,-139.0333,78.67,63,6,12.17
14,14,Tiznit,MA,2022-07-19 22:00:25,overcast clouds,29.5833,-9.5000,87.71,30,86,0.51
18,18,Jamestown,US,2022-07-19 21:57:27,few clouds,42.0970,-79.2353,86.95,61,21,3.00
22,22,Manggar,ID,2022-07-19 21:56:20,scattered clouds,-2.8833,108.2667,80.24,85,50,11.23
24,24,Butaritari,KI,2022-07-19 21:57:31,light rain,3.0707,172.7902,82.15,76,74,15.73
28,28,Sompeta,IN,2022-07-19 22:00:32,overcast clouds,18.9333,84.6000,83.12,79,86,5.91
36,36,Avarua,CK,2022-07-19 21:57:31,few clouds,-21.2078,-159.7750,77.05,65,20,9.22
37,37,Tautira,PF,2022-07-19 22:00:35,few clouds,-17.7333,-149.1500,76.35,68,11,2.95


In [5]:
# 4a. Determine if there are any empty rows.
# determine if there are any null values.
preferred_cities_df.count()


City_ID                221
City                   221
Country                221
Date                   221
Current Description    221
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.dropna(how="any", inplace=True)



In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,
9,Tiarei,PF,76.46,clear sky,-17.5333,-149.3333,
13,Atuona,PF,78.67,clear sky,-9.8000,-139.0333,
14,Tiznit,MA,87.71,overcast clouds,29.5833,-9.5000,
18,Jamestown,US,86.95,few clouds,42.0970,-79.2353,
22,Manggar,ID,80.24,scattered clouds,-2.8833,108.2667,
24,Butaritari,KI,82.15,light rain,3.0707,172.7902,
28,Sompeta,IN,83.12,overcast clouds,18.9333,84.6000,
36,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,
37,Tautira,PF,76.35,few clouds,-17.7333,-149.1500,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
     
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [ ]:
# get map
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig